In [2]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing import *

In [3]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

170498071/170498071 [==============================] - 12s 0us/step


In [4]:
from keras import backend as K

# Swish Activation Function
def swish(x):
    return K.sigmoid(x) * x

# Swish Layer
class Swish(Dense):
    def __init__(self, activation=swish, **kwargs):
        super(Swish, self).__init__(**kwargs)
        self.activation = activation

In [9]:
def dense_block(x, blocks, growth_rate):
    for i in range(blocks):
        x = conv_block(x, growth_rate)
    return x

def conv_block(x, growth_rate):
    x1 = BatchNormalization()(x)
    
    
    
    x1 = Activation(swish)(x1)
    
    
    
    x1 = Conv2D(4 * growth_rate, (1, 1), use_bias=False, padding='same')(x1)
    x1 = BatchNormalization()(x1)
    
    
    
    x1 = Activation(swish)(x1)
    
    
    
    x1 = Conv2D(growth_rate, (3, 3), use_bias=False, padding='same')(x1)
    x = tf.keras.layers.Concatenate()([x, x1])
    return x

def transition_block(x, reduction):
    x = BatchNormalization()(x)
    
    
    
    x = Activation(swish)(x)
    
    
    
    x = Conv2D(int(tf.keras.backend.int_shape(x)[3] * reduction), (1, 1), use_bias=False, padding='same')(x)
    x = AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x

def densenet(blocks=[6, 12, 64, 48], growth_rate=32, include_top=True, weights=None, input_shape=None, classes=1000):
    inputs = Input(shape=input_shape)
    x = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(inputs)
    x = Conv2D(64, (7, 7), strides=(2, 2), use_bias=False)(x)
    x = BatchNormalization()(x)
    
    
    
    x = Activation(swish)(x)
    
    
    
    x = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    for i, num_blocks in enumerate(blocks):
        x = dense_block(x, num_blocks, growth_rate)
        if i != len(blocks) - 1:
            x = transition_block(x, 0.5)

    if include_top:
        x = GlobalAveragePooling2D()(x)
        x = Dense(classes, activation='softmax')(x)
        
    model = Model(inputs, x, name='densenet')
    return model

model = densenet(input_shape=(32,32,3), classes=10)
model.summary()

Model: "densenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 38, 38, 3)   0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_264 (Conv2D)            (None, 16, 16, 64)   9408        ['zero_padding2d_2[0][0]']       
                                                                                                  
 batch_normalization_264 (Batch  (None, 16, 16, 64)  256         ['conv2d_264[0][0]']      

In [10]:
from tensorflow.keras import callbacks
class WandbCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log(logs)


# Define data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
)


In [11]:
wandb.init(project="CIFAR", entity="hcim", name='DenseNet_Swish')

In [12]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=[CategoricalAccuracy()])

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=100, 
                    validation_data=(x_test, y_test), callbacks=[WandbCallback()])

Epoch 1/100
782/782 [==============================] - 96s 100ms/step - loss: 1.7803 - categorical_accuracy: 0.3923 - val_loss: 2.0070 - val_categorical_accuracy: 0.3454
Epoch 2/100
782/782 [==============================] - 76s 97ms/step - loss: 1.3106 - categorical_accuracy: 0.5287 - val_loss: 2.1436 - val_categorical_accuracy: 0.3964
Epoch 3/100
782/782 [==============================] - 76s 97ms/step - loss: 1.1681 - categorical_accuracy: 0.5805 - val_loss: 1.7051 - val_categorical_accuracy: 0.4890
Epoch 4/100
782/782 [==============================] - 76s 97ms/step - loss: 1.0098 - categorical_accuracy: 0.6417 - val_loss: 1.3554 - val_categorical_accuracy: 0.5563
Epoch 5/100
782/782 [==============================] - 76s 97ms/step - loss: 1.1827 - categorical_accuracy: 0.5846 - val_loss: 2.1690 - val_categorical_accuracy: 0.4107
Epoch 6/100
782/782 [==============================] - 76s 97ms/step - loss: 0.9351 - categorical_accuracy: 0.6698 - val_loss: 0.9520 - val_categorical_ac

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



782/782 [==============================] - 75s 96ms/step - loss: 0.4718 - categorical_accuracy: 0.8342 - val_loss: 3.9368 - val_categorical_accuracy: 0.7434
Epoch 82/100
782/782 [==============================] - 75s 96ms/step - loss: 0.8620 - categorical_accuracy: 0.6997 - val_loss: 0.8772 - val_categorical_accuracy: 0.7164
Epoch 83/100
782/782 [==============================] - 75s 96ms/step - loss: 0.6939 - categorical_accuracy: 0.7575 - val_loss: 0.8327 - val_categorical_accuracy: 0.7578
Epoch 84/100
782/782 [==============================] - 75s 96ms/step - loss: 0.6020 - categorical_accuracy: 0.7901 - val_loss: 1.0258 - val_categorical_accuracy: 0.7732
Epoch 85/100
782/782 [==============================] - 75s 96ms/step - loss: 0.5578 - categorical_accuracy: 0.8056 - val_loss: 1.3398 - val_categorical_accuracy: 0.7754
Epoch 86/100
782/782 [==============================] - 76s 97ms/step - loss: 0.5206 - categorical_accuracy: 0.8193 - val_loss: 0.7383 - val_categorical_accuracy: 